In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [13]:
using LinearAlgebra
using ForwardDiff
using Printf

In [10]:
function discrete_dynamics(params,x,u,i)
  # RK4
  A = [1.0  0.0  0.1  0.0;
       0.0  1.0  0.0  0.1;
       0.0  0.0  1.0  0.0;
       0.0  0.0  0.0  1.0]
  B = [0.005  0.0;
       0.0    0.005;
       0.1    0.0;
       0.0    0.1]
  return A*x + B*u
end

discrete_dynamics (generic function with 2 methods)

In [28]:
include("tinyMPC.jl")
nx = 4
nu = 2
N = 101
dt = 0.01
t_vec = dt*(0:N-1)

x0 = [5, 7, 2, -1.4]
xg = [2, 5, 3, -1.]

Uref = [zeros(nu) for k = 1:N-1]
Xref = [deepcopy(xg) for k = 1:N]

u_min = [-1;-1]
u_max = [1; 1]

x_min = -100.0*ones(nx)
# x_min[1:7] .= -.8
x_max = 100.0*ones(nx)
# x_max[1:7] .= 1.0

# Cost weights
Q = 0.1*I(nx)
R = 0.1*I(nu)
Qf = 0.1*I(nx)

max_inner_iters = 10  # MAX RICCATI SOLVING ITERATION
ncx = 2*nx*0
ncu = 2*nu*0
ncg = 0
ncu_cone = nu*0; 
cone_scale = 1e-3  # very fragile for each problem

μ = [zeros(ncu) for i = 1:N-1]   # input constraints
μx = [zeros(ncx) for i = 1:N]    # state constraints
λ = zeros(nx)    # goal constraint
λc = [zeros(ncu_cone) for i = 1:N-1]   # goal constraint
A_cone = [1 0 0; 0 1 0]
c_cone = [0.0, 0.0, tand(0)]

params = (
    nx = nx, nu = nu, ncx = ncx, ncu = ncu, ncg = ncg, ncu_cone = ncu_cone,
    μ = μ, μx = μx, λ = λ, λc = λc,
    A_cone = A_cone, c_cone = c_cone,
    N = N, Q = Q, R = R, Qf = Q,
    u_min = u_min, u_max = u_max,
    x_min = x_min, x_max = x_max,
    Xref = Xref, Uref = Uref, dt = dt,
);

# previous MPC iterate
X = [zeros(nx) for i = 1:N]
U = [ones(nu) for k = 1:N-1]
# new MPC iterate
Xn = deepcopy(X)
Un = deepcopy(U)

P = [zeros(nx,nx) for i = 1:N]   # cost to go quadratic term
p = [zeros(nx) for i = 1:N]      # cost to go linear term
d = [zeros(nu) for i = 1:N-1]    # feedforward control
K = [zeros(nu,nx) for i = 1:N-1] # feedback gain

X[1] .= x0;
u = mpc!(params,X,U,P,p,K,d,Xn,Un; atol=1e-4,max_iters=20,verbose=true,ρ=1e0,ρ_max=1e10, ϕ=10.0)

ΔJ = 0.0
iter     J           ΔJ        |d|         α        reg         ρ
---------------------------------------------------------------------
  1    7.244e+01   0.00e+00   0.00e+00  1.0000    1.00e-08    1.00e+00
convio = 0


2-element Vector{Float64}:
 -6.023555824561335
  0.4560657907663863

In [31]:
X

101-element Vector{Vector{Float64}}:
 [5.0, 7.0, 2.0, -1.4]
 [5.169882220877193, 6.862280328953832, 1.3976444175438665, -1.3543934209233612]
 [5.283675126489349, 6.72938012091104, 0.8782136946992644, -1.3036107399324817]
 [5.349250148502516, 6.601752585892297, 0.4332867455640642, -1.2489399604423845]
 [5.37366937768119, 6.4797305908195995, 0.05509783800941792, -1.1914999410115708]
 [5.3632494507414945, 6.363542742617317, -0.26349637680331284, -1.132257023034088]
 [5.3236221168795055, 6.2533278707657685, -0.5290503004364664, -1.07204041399688]
 [5.259791456462173, 6.14914803091011, -0.747562907910185, -1.0115563831162662]
 [5.17618775946237, 6.051000145352583, -0.9245110320858768, -0.9514013280342783]
 [5.076718100667413, 5.958826390336458, -1.0648821438132612, -0.8920737722882279]
 ⋮
 [3.426901624439806, 4.522978333060275, 1.1713614448669283, -0.38858609374238967]
 [3.5469512774928247, 4.4831431929794014, 1.2296316161934469, -0.40811670787507]
 [3.672712352630024, 4.441394170994326, 1.

In [33]:
d

100-element Vector{Vector{Float64}}:
 [-1.833008206989337, -4.585752231657667]
 [-1.832840856037217, -4.58580784708591]
 [-1.832656133323719, -4.585869221302517]
 [-1.8324527686568746, -4.585936771690476]
 [-1.8322294548787217, -4.586010926816633]
 [-1.8319848551039912, -4.586092123803354]
 [-1.8317176116460592, -4.586180805097012]
 [-1.8314263568508238, -4.586277414552]
 [-1.83110972607984, -4.586382392740679]
 [-1.830766373106048, -4.586496171390605]
 ⋮
 [-2.593345325146418, -0.9255451154266795]
 [-2.3789117067252428, -0.688874477646207]
 [-2.13464077477255, -0.4717862481836722]
 [-1.8640571582308796, -0.2811363927723023]
 [-1.5718233980145757, -0.12347710596098295]
 [-1.263536885279889, -0.004723847762237438]
 [-0.9454761149434475, 0.07013844110947785]
 [-0.6243278545364681, 0.09729149969859835]
 [-0.30692309596297124, 0.07425558773297691]